In [1]:
from tqdm import tqdm_notebook as tqdm

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import os
from visualization import highlight_sentences

In [3]:
import pyrouge

In [4]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

## Data

Будем использовать данные из корпуса новостей CNN/DailyMail.
В рамках семинара используется подвыборка из 300 текстов CNN.

In [6]:
DATA_DIR = './cnn_stories_short/'

In [7]:
!wget https://www.dropbox.com/s/kofxrgod7kl720m/cnn_stories_short.zip
!mkdir cnn_data 
!unzip cnn_stories_short.zip -d $DATA_DIR

--2018-12-05 09:39:00--  https://www.dropbox.com/s/kofxrgod7kl720m/cnn_stories_short.zip
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.66.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/kofxrgod7kl720m/cnn_stories_short.zip [переход]
--2018-12-05 09:39:02--  https://www.dropbox.com/s/raw/kofxrgod7kl720m/cnn_stories_short.zip
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://ucc23f13e6a28dbaeae950b2809d.dl.dropboxusercontent.com/cd/0/inline/AW27OapwaPGf1PXE7miQ5g8tuUSypCsdDbwQYEPIWBvJV1zwSSajA0ZW00Hco3MDEyySQM7qfWII-qATuvmSbBq_3SWr_0cOasaoHmH8-REu31JSl-kqBbiLDWQeSOjEPYet2RYYZsrCtCSJoEoFFvm_RE2kqLRq8zC60Yd7D4d2Lqc3HLz8UKFBTTlmF8fkuC4/file [переход]
--2018-12-05 09:39:02--  https://ucc23f13e6a28dbaeae950b2809d.dl.dropboxusercontent.com/cd/0/inline/AW27OapwaPGf1PXE7miQ5g8tuUSypCsdDbwQYEP

## Data preparation

In [8]:
texts = []
summaries = []
for filename in os.listdir(DATA_DIR):
    with open(os.path.join(DATA_DIR,filename),'r') as input_file:
        all_texts = input_file.read().split('@highlight')
        texts.append(all_texts[0])
        summaries.append('. '.join(map(lambda x: x.strip(), all_texts[1:])))

#### Нам понадобятся: 
* тексты, разбитые на предложения 
* предложения, разбитые на токены
* тексты, разбитые предложения, которые разбиты на токены

In [ ]:
sent_tokenized_texts = [sent_tokenize(text) for text in texts]
tokenized_sentences = [word_tokenize(sent) for text in texts for sent in sent_tokenize(text)]
tokenized_texts = [[word_tokenize(sent) for sent in text] for text in sent_tokenized_texts]

## Word Embeddings

Будем использовать предобученные вектора Glove. 

Загрузим модель:

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2018-12-05 09:41:57--  http://nlp.stanford.edu/data/glove.6B.zip
Распознаётся nlp.stanford.edu (nlp.stanford.edu)… 171.64.67.140
Подключение к nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://nlp.stanford.edu/data/glove.6B.zip [переход]
--2018-12-05 09:41:57--  https://nlp.stanford.edu/data/glove.6B.zip
Подключение к nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 862182613 (822M) [application/zip]
Сохранение в: «glove.6B.zip.1»

glove.6B.zip.1      100%[===================>] 822,24M  3,34MB/s    за 3m 34s  

2018-12-05 09:45:32 (3,85 MB/s) - «glove.6B.zip.1» сохранён [862182613/862182613]

Archive:  glove.6B.zip
replace glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
word_embeddings = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f.readlines():
        values = line.split()
        word = values[0]
        word_embeddings[word] = np.asarray(values[1:], dtype='float32')

На основе эмбеддингов слов строим векторные представления предложений:

In [ ]:
class TfidfEmbeddingVectorizer:
    
    def __init__(self, embedding_model, dim=100):
        self.embedding_model = embedding_model
        self.word2weight = None
        self.dim = dim

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = np.max(tfidf.idf_)
        self.word2weight = defaultdict(lambda: max_idf, [(w,tfidf.idf_[i]) for w,i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([np.mean([self.embedding_model[w] * self.word2weight[w] 
                                  for w in words if w in self.embedding_model] or [np.zeros(self.dim)], axis=0) 
                         for words in X])

In [ ]:
sentence_vectorizer = TfidfEmbeddingVectorizer(word_embeddings)
sentence_vectorizer = sentence_vectorizer.fit(tokenized_sentences)

## Similarity matrix

Выберем один текст и построим для него матрицу расстояний. В качестве метрики используем косинусное расстояние.

In [ ]:
TEXT_NUM = 5

In [ ]:
sentences = tokenized_texts[TEXT_NUM]

In [ ]:
vectorized_sentences = sentence_vectorizer.transform(sentences)

In [ ]:
G = cosine_similarity(vectorized_sentences)

## Extractive Summarization $-$ TextRank

$$ G = (V,E) - граф $$
$$$$
$$ PageRank(w) = (1-d) +  d \sum_{u} \frac {PageRank(u)} {C(u)}$$

$$u\ -\ вершина\ графа,\ такая\ что\ (u,w) \in E$$
$$$$
$$d = 0,85\ -\ коэффициент\ затухания$$

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

In [ ]:
def page_rank(G, s = .85, maxerr = .0001):
    
    n = G.shape[0]
    A = csr_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    sink = rsums==0
    ro, r = np.zeros(n), np.ones(n)
    while np.sum(np.abs(r-ro)) > maxerr:
        ro = r.copy()
        for i in range(0,n):
            ## your
            ## code
            ## here
    return r/float(sum(r))

In [ ]:
scores = pageRank(G)

Сравним нашу реализацию с реализацией NetworkX

In [ ]:
#!pip install networkx

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_matrix(G)
nx_scores = nx.pagerank(nx_graph)

In [ ]:
TEXT_NUM
print("Our implementation: {0}\nNetworkX implementation: {1}".format(scores[sentence_num],nx_scores[sentence_num]))

In [ ]:
ranked_sentences = sorted(((scores[i],s,i) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
SUMMARY_LEN  = 5

for i in range(SUMMARY_LEN):
    print(' '.join(ranked_sentences[i][1]))

In [ ]:
extracted_sentences = [sent_tokenized_texts[test_sentence_num][i] for score,sentence,i in ranked_sentences][:5]

In [ ]:
def make_html(sent_tokenized_text,extracted_sentences):
    result_html = '<body>'
    for sentence in sent_tokenized_text:
        if sentence in extracted_sentences:
            result_html += '<p class="highlighted_text"> {0}.</p>'.format(sentence.replace('\n', '<br>'))
        else:
            result_html += ' {0}'.format(sentence)#'<p class="raw_text"> {0}.</p>'.format(sentence.replace('\n', '<br>'))
    return result_html + '</body>'

In [ ]:
highlight_sentences(sent_tokenized_texts[5],extracted_sentences)
css_styling()

## Метрика качества:

$Доля\ n-грамм\ из\ рефератов,\ вошедших\ в\ s:$
$$$$
$ ROUGE_n(S) = \frac{\sum_{r\in R} \sum_{w} [w \in s][w \in r]}{\sum_{r \in R} \sum_w [w \in r]}$ 
$$$$
$ Доля\ n-грамм\ самого\ близкого\ реферата,\ вошедших\ в\ s:$
$$$$
$ ROUGE_{n_{multi}}(S) = \frac{max_{r\in R} \sum_{w} [w \in s][w \in r]}{\sum_{r \in R} \sum_w [w \in r]}$